In [1]:
using JuMP
using GLPK
using LinearAlgebra,Cbc
using Xpress

┌ Info: Xpress: Found license file C:\xpressmp\bin\xpauth.xpr
└ @ Xpress C:\Users\tjfur\.julia\packages\Xpress\eJoYN\src\license.jl:44
┌ Info: Xpress: Development license detected.
└ @ Xpress C:\Users\tjfur\.julia\packages\Xpress\eJoYN\src\license.jl:89


Question 1

## Answer
Let the ith item have a utility $u_i$ dollars and weight $w_i$ Kg. The hiker wants to select a subset with maximum utility, such that the summation of the weights of the selected items is less or equal to 15 Kg and I must pick atleast three items.
 
Considering a set of decision binary variables $𝑥_𝑖$ that receive value 1 if the $𝑖-th$ item is selected, and 0 if it is not selected, the resulting mathematical programming formulation is:

$$Maximise\ \sum_{i\in I}u_i.x_i$$
$$s.t.\ \sum_{i\in I}w_i.x_i \le W$$
$$x_i\in\{0,1\}\ \ \ \forall i\in I$$

In [8]:
using JuMP
import GLPK
import Test

profit = [4, 2, 1, 7, 3, 6]
weight = [5, 8, 8, 6, 1, 5]
capacity = 15
model = Model(Xpress.Optimizer)
@variable(model, x[1:6], Bin)
# Objective: maximize profit
@objective(model, Max, profit' * x)
# Constraint: can carry all
@constraint(model, weight' * x <= capacity)
# Constraint must pick three items
@constraint(Q1, (sum(x[i] for i=1:6)) >= 3
# Solve problem using MIP solver
optimize!(model)

println("The best utility the hiker can get is: ", objective_value(model))
println("Solution is:")

for i in 1:6
    print("x[$i] = ", value(x[i]))
    println(", p[$i]/w[$i] = ", profit[i] / weight[i])
end

FICO Xpress v8.12.3, Community, solve started 16:06:00, Nov 30, 2021
Heap usage: 81KB (peak 81KB, 955KB system)
Maximizing MILP  with these control settings:
OUTPUTLOG = 1
MPSNAMELENGTH = 64
CALLBACKFROMMASTERTHREAD = 1
Original problem has:
         1 rows            6 cols            6 elements         6 globals
Presolved problem has:
         1 rows            6 cols            6 elements         6 globals
Presolve finished in 0 seconds
Heap usage: 107KB (peak 120KB, 957KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  8.00e+00] / [ 1.25e-01,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.50e+01] / [ 1.00e+00,  1.88e+00]
  Objective      [min,max] : [ 1.00e+00,  7.00e+00] / [ 1.00e+00,  7.00e+00]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 4.8GB
 *** Solution found:      .000000   Time:   0    Heuristic: T ***
 *** Solution found:    16.000000 

# Question 2
Consider the following binary integer program.

$$ Maximise\ 12x_1+x_2+10x_3+2x_4 $$
$$s.t.$$
$$ 16x_1+2x_2+10x_3+8x_4\le 18 $$
$$x_i\ is\ binary\ for\ j=1,...,4$$

Use the branch-and-bound method to find an optimal solution to this problem. You can use Julia/JuMP and a solver to solve the LP relaxations of the subproblems resulting from fixing some decision variables at 0 or 1. 

Please do not use the Julia/JuMP and the solver to directly solve the ILP itself.

# Answer

Let's start by fixing the variable $x_1$.
Fixing $x_1=0$
$$ Maximise\ \ \ x_2+10x_3+2x_4 $$
$$s.t.$$
$$ \ \ 2x_2+10x_3+8x_4\le 18 $$
$$0\le x_i\le 1\ for\ j=\ \ 2,...,4$$

In [15]:
model = Model(GLPK.Optimizer)  
@variable(model, x1==0)
@variable(model, 0 <= x2 <= 1)
@variable(model, 0 <= x3 <= 1 )
@variable(model, 0 <= x4 <= 1 )
# Constraints
# @constraint(model, x2 >= 0)
#@constraint(model, x2 <=0)
@constraint(model, 16x1 + 2x2 + 10x3 + 8x4 <= 18)

# Objective Function:
@objective(model, Max, 12x1 + x2 + 10x3 + 2x4)
optimize!(model)

if termination_status(model) == MOI.OPTIMAL
    println("x1-x4 value: $(value(x1)), $(value(x2)), $(value(x3)), $(value(x4))")
    println("objective value: ",objective_value(model))
elseif termination_status(model) == MOI.INFEASIBLE
    println("The model is infeasible")
else
    println("The model didn't solve properly for some other reason.")
end

x1-x4 value: 0.0, 1.0, 1.0, 0.75
objective value: 12.5


Lets explore when we set x1 = 1

In [16]:
model = Model(GLPK.Optimizer)  
@variable(model, x1==1)
@variable(model, 0 <= x2 <= 1)
@variable(model, 0 <= x3 <= 1 )
@variable(model, 0 <= x4 <= 1 )
# Constraints
# @constraint(model, x2 >= 0)
#@constraint(model, x2 <=0)
@constraint(model, 16x1 + 2x2 + 10x3 + 8x4 <= 18)

# Objective Function:
@objective(model, Max, 12x1 + x2 + 10x3 + 2x4)
optimize!(model)

if termination_status(model) == MOI.OPTIMAL
    println("x1-x4 value: $(value(x1)), $(value(x2)), $(value(x3)), $(value(x4))")
    println("objective value: ",objective_value(model))
elseif termination_status(model) == MOI.INFEASIBLE
    println("The model is infeasible")
else
    println("The model didn't solve properly for some other reason.")
end

x1-x4 value: 1.0, 0.0, 0.2, 0.0
objective value: 14.0


Since this is a maximisation problem, LP gave us a upper bound of 14 when $x1 = 1$. This branch needs to be explored further by fixing other variables. Let's fix x2 = 0 and x2 = 1.

Fixing $x_1=1, x_2=0$
$$ Maximise\ \ \ 12 + 10x_3+2x_4 $$
$$s.t.$$
$$ \ \ 16+10x_3+8x_4\le 18 $$
$$0\le x_i\le 1\ for\ j=\ \ 3,4$$

In [17]:
model = Model(GLPK.Optimizer)  
@variable(model, x1==1)
@variable(model, x2==0)
@variable(model, 0 <= x3 <= 1 )
@variable(model, 0 <= x4 <= 1 )
# Constraints
# @constraint(model, x2 >= 0)
#@constraint(model, x2 <=0)
@constraint(model, 16x1 + 2x2 + 10x3 + 8x4 <= 18)

# Objective Function:
@objective(model, Max, 12x1 + x2 + 10x3 + 2x4)
optimize!(model)

if termination_status(model) == MOI.OPTIMAL
    println("x1-x4 value: $(value(x1)), $(value(x2)), $(value(x3)), $(value(x4))")
    println("objective value: ",objective_value(model))
elseif termination_status(model) == MOI.INFEASIBLE
    println("The model is infeasible")
else
    println("The model didn't solve properly for some other reason.")
end

x1-x4 value: 1.0, 0.0, 0.2, 0.0
objective value: 14.0


Fixing $x_1=1, x_2=1$
$$ Maximise\ \ \ 12+1+10x_3+2x_4 $$
$$s.t.$$
$$ \ \ 16+2+10x_3+8x_4\le 18 $$
$$0\le x_i\le 1\ for\ j=\ \ 3,4$$

In [18]:
model = Model(GLPK.Optimizer)  
@variable(model, x1==1)
@variable(model, x2==1)
@variable(model, 0 <= x3 <= 1 )
@variable(model, 0 <= x4 <= 1 )
# Constraints
# @constraint(model, x2 >= 0)
#@constraint(model, x2 <=0)
@constraint(model, 16x1 + 2x2 + 10x3 + 8x4 <= 18)

# Objective Function:
@objective(model, Max, 12x1 + x2 + 10x3 + 2x4)
optimize!(model)

if termination_status(model) == MOI.OPTIMAL
    println("x1-x4 value: $(value(x1)), $(value(x2)), $(value(x3)), $(value(x4))")
    println("objective value: ",objective_value(model))
elseif termination_status(model) == MOI.INFEASIBLE
    println("The model is infeasible")
else
    println("The model didn't solve properly for some other reason.")
end

x1-x4 value: 1.0, 1.0, 0.0, 0.0
objective value: 13.0


when $x2=0$ the LP gave us an upper bound of 14, but we cannot have decimals, unfortunately $x3$ was 0.2 so we explore this branch when $x2=1$ by fixing the $x3=1, x3=0$

Fixing $x_1=1, x_2=1, x_3=0$
$$ Maximise\ \ \ 12 + 1 + 2x_4 $$
$$s.t.$$
$$ \ \ 16 + 2 + 8x_4\le 18 $$
$$0\le x_i\le 1\ for\ j=\ \ 4$$

In [3]:
model = Model(GLPK.Optimizer)  
@variable(model, x1==1)
@variable(model, x2==1)
@variable(model, x3==0)
@variable(model, 0 <= x4 <= 1 )
# Constraints
# @constraint(model, x2 >= 0)
#@constraint(model, x2 <=0)
@constraint(model, 16x1 + 2x2 + 10x3 + 8x4 <= 18)

# Objective Function:
@objective(model, Max, 12x1 + x2 + 10x3 + 2x4)
optimize!(model)

if termination_status(model) == MOI.OPTIMAL
    println("x1-x4 value: $(value(x1)), $(value(x2)), $(value(x3)), $(value(x4))")
    println("objective value: ",objective_value(model))
elseif termination_status(model) == MOI.INFEASIBLE
    println("The model is infeasible")
else
    println("The model didn't solve properly for some other reason.")
end

x1-x4 value: 1.0, 1.0, 0.0, 0.0
objective value: 13.0


Fixing $x_1=1, x_2=0, x_3=1$
$$ Maximise\ \ \ 12 + 10 + 2x_4 $$
$$s.t.$$
$$ \ \ 16 + 10 + 8x_4\le 18 $$
$$0\le x_i\le 1\ for\ j=\ \ 4$$

In [5]:
model = Model(GLPK.Optimizer)  
@variable(model, x1==1)
@variable(model, x2==1)
@variable(model, x3==1)
@variable(model, 0 <= x4 <= 1 )
# Constraints
# @constraint(model, x2 >= 0)
#@constraint(model, x2 <=0)
@constraint(model, 16x1 + 2x2 + 10x3 + 8x4 <= 18)

# Objective Function:
@objective(model, Max, 12x1 + x2 + 10x3 + 2x4)
optimize!(model)

if termination_status(model) == MOI.OPTIMAL
    println("x1-x4 value: $(value(x1)), $(value(x2)), $(value(x3)), $(value(x4))")
    println("objective value: ",objective_value(model))
elseif termination_status(model) == MOI.INFEASIBLE
    println("The model is infeasible")
else
    println("The model didn't solve properly for some other reason.")
end

The model is infeasible


This results in an infeasible solution, the branch $x3=1$ can be fathomed.Lets explore when we fix $x4=1, x4=0$

Fixing $x_1=1, x_2=1, x_3=0, x_4=0$
$$ Maximise\ \ \ 12  $$
$$s.t.$$
$$ \ \ 16 \le 18 $$
$$0\le x_i\le 1\ for\ j=\ \ 4$$

In [7]:
model = Model(GLPK.Optimizer)  
@variable(model, x1==1)
@variable(model, x2==1)
@variable(model, x3==0)
@variable(model, x4==0)
# Constraints
# @constraint(model, x2 >= 0)
#@constraint(model, x2 <=0)
@constraint(model, 16x1 + 2x2 + 10x3 + 8x4 <= 18)

# Objective Function:
@objective(model, Max, 12x1 + x2 + 10x3 + 2x4)
optimize!(model)

if termination_status(model) == MOI.OPTIMAL
    println("x1-x4 value: $(value(x1)), $(value(x2)), $(value(x3)), $(value(x4))")
    println("objective value: ",objective_value(model))
elseif termination_status(model) == MOI.INFEASIBLE
    println("The model is infeasible")
else
    println("The model didn't solve properly for some other reason.")
end

x1-x4 value: 1.0, 1.0, 0.0, 0.0
objective value: 13.0


Fixing $x_1=1, x_2=1, x_3=0 x_4=1$
$$ Maximise\ \ \ 12 +2$$
$$s.t.$$
$$ \ \ 16 + 8\le 18 $$
$$0\le x_i\le 1\ for\ j=\ \ 4$$

In [8]:
model = Model(GLPK.Optimizer)  
@variable(model, x1==1)
@variable(model, x2==1)
@variable(model, x3==0)
@variable(model, x4==1)
# Constraints
# @constraint(model, x2 >= 0)
#@constraint(model, x2 <=0)
@constraint(model, 16x1 + 2x2 + 10x3 + 8x4 <= 18)

# Objective Function:
@objective(model, Max, 12x1 + x2 + 10x3 + 2x4)
optimize!(model)

if termination_status(model) == MOI.OPTIMAL
    println("x1-x4 value: $(value(x1)), $(value(x2)), $(value(x3)), $(value(x4))")
    println("objective value: ",objective_value(model))
elseif termination_status(model) == MOI.INFEASIBLE
    println("The model is infeasible")
else
    println("The model didn't solve properly for some other reason.")
end

The model is infeasible


When $x4=1$ the LP is infeasible so we can fathom this branch.

We have fathomed all infeasible branches. Thus we conclude that the objective value is 13.0 and the optimal solution is $x1, x2 = 1, x3, x4 = 0$.

# Question 3

# Answer

In [2]:
model = Model(GLPK.Optimizer)
# variables are Binary variables to represent wether a node x_i is in the 
# minimum vertex cover set S_1 (0 = node not in S_1, 1 = node in S_1)
@variable(model, x[1:6], Bin)
@objective(model, Min, sum(x))

#constraints:
#Every edge in this graph needs to be considered as a constraint 
tuples = [(x[1], x[2]), 
    (x[2], x[2]), 
    (x[3], x[1]), 
    (x[3], x[4]), 
    (x[5], x[4]), 
    (x[6], x[5]), 
    (x[6], x[4])]
#To cover all edges (u, v) ∈ E on a graph, we need either u or v in the vertex cover set
#Or gate = sum of vertices x_i + x_j >= 1.
for edge in tuples
    @constraint(model, sum(edge) >= 1)
end

optimize!(model)
println("The minimum cardinality vertex cover for this graph is: ", objective_value(model))
println("nodes in S1 are : ")
    
for i in 1:6
    if value(x[i]) >= 1
        println("x_$i = ", value(x[i]))
    end
end

The minimum cardinality vertext cover for this graph is: 4.0
nodes in S1 are : 
x_2 = 1.0
x_3 = 1.0
x_4 = 1.0
x_6 = 1.0
